In [17]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import losses
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import relu,softmax,sigmoid
from tensorflow.keras.optimizers import Adam
import os
import mysql.connector
import pandas as pd

DataBaseden veri çekmek.
Environment yüklemek.

In [ ]:
from dotenv import load_dotenv  # python-dotenv kütüphanesini import edin

load_dotenv("/Users/enesdemir/Desktop/financeProject/.venv/.env")

In [3]:
password = os.getenv("MYSQL_PASSWORD")

db = mysql.connector.connect(
    host="localhost",
    user="root",  
    password=f"{password}",
    database="finance"  
)

cursor = db.cursor()

In [4]:
query = """
    SELECT timestampp,open_price,open_price,high_price,low_price,close_price,volume FROM stock_data;
"""

cursor.execute(query)
answer = cursor.fetchall()

Veriyi numpy dizisine çevirme.

In [37]:
train_x =[]
train_y =[]

for row in answer:
    row_data = np.array([
        row[0].timestamp(),
        float(row[1]),
        float(row[2]),
        float(row[3]),
        float(row[4]),
        float(row[5]),
        int(row[6])
    ])
    row_data2 = np.array([

    ])
    train_x.append(row_data)

train_x = pd.DataFrame(train_x)


In [39]:
print(train_x)

                 0         1         2         3         4         5       6
0     1.737651e+09  225.8800  225.8800  226.0000  225.8800  226.0000    94.0
1     1.737651e+09  225.8898  225.8898  225.8898  225.8800  225.8800   203.0
2     1.737650e+09  225.8822  225.8822  226.0000  225.8733  225.9000    45.0
3     1.737650e+09  226.0000  226.0000  226.0000  225.9997  225.9997    10.0
4     1.737650e+09  226.0000  226.0000  226.0400  226.0000  226.0400  1355.0
...            ...       ...       ...       ...       ...       ...     ...
3156  1.735176e+09  224.5700  224.5700  224.5900  223.2800  224.5900    19.0
3157  1.735176e+09  224.5900  224.5900  224.6500  224.1900  224.1900    39.0
3158  1.735175e+09  224.5900  224.5900  224.5900  223.9700  224.5900   221.0
3159  1.735175e+09  224.4100  224.4100  224.7000  224.0200  224.5900    47.0
3160  1.735175e+09  224.6000  224.6000  224.6000  223.9000  224.4100   355.0

[3161 rows x 7 columns]


In [44]:
train_x.ffill(inplace=True)  